In [1]:
from susy_qm import ansatze
from susy_qm.ansatze import gate_list_from_ansatz
import pennylane as qml
from pennylane import numpy as np

import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

import pandas as pd
import json
import os

In [2]:
def label_to_gate_count(label):
    parts = label.split("_")
    total = parts[-1]
    return int(total)

In [4]:
potentials = ['AHO','DW']
cutoffs = [2,4,8,16,32,64]

for potential in potentials:
    for cutoff in cutoffs:

        print(f"Running for {potential} potential and {str(cutoff)} cutoff")

        folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\AnsatzSteps\10000\{}\{}".format(potential,cutoff))
        folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

        num_qubits = int(np.log2(cutoff)+1) 
        ansatz_name = f"CQAVQE_{potential}{cutoff}_exact"
        ansatz = ansatze.get(ansatz_name)
        num_params = ansatz.n_params

        dev = qml.device("default.qubit", wires=num_qubits)
        @qml.qnode(dev)
        def circuit(params):
            ansatz(params, num_qubits)
            return qml.state()

        params = np.array([0]*num_params)
        gates = gate_list_from_ansatz(ansatz, params, num_qubits)

        gate_list = {}
        op_labels = ""
        counter = 1
        for item in gates:
            if item["gate"] == "BasisState":
                continue

            gate = item["gate"]
            wires = item["wires"]

            qiskit_wires = [f"$q_{(num_qubits - 1) - w}$" for w in wires]
            label = f"{gate}[{', '.join(qiskit_wires)}]"

            if counter == 1:
                op_labels = label
            else:
                op_labels = op_labels + ", " + label

            gate_list[str(counter)] = op_labels
            counter +=1

        avqe_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles\{}\data_{}.txt".format(potential,cutoff))

        with open(avqe_path, "r", encoding="utf-8") as f:
            text = f.read()

        json_part = text[: text.rfind("}") + 1]
        data = json.loads(json_part)

        energy_diffs = data["energy_diffs"]
        energy_changes = data["energy_change"]
        step_energies = data["best_energy_list_reduced"]

        all_data = []

        for f in folders:

            gate_count = label_to_gate_count(f)

            data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
            
            with open(data_path, 'r') as file:
                data = json.load(file)

            success = sum(data['success'])
            exact_e = np.min(data['exact_eigenvalues'])
            converged_runs = sum(data['success'])
            mean_evals = int(np.round(np.mean(data['num_evaluations'])))

            min_e = np.min(data['results'])
            delta_min_e = abs(exact_e - np.min(data['results']))
            median_e = np.median(data['results'])
            delta_median_e = abs(exact_e - np.median(data['results']))
            
            time_taken = data['total_VQE_time']

            ansatz_step = gate_list[str(gate_count)]

            data_dict = {"gate list": ansatz_step, 
                        "avqe step": gate_count,
                        'Converged Runs': f"{converged_runs}/{len(data['results'])}", 
                        "step_energies": step_energies[gate_count-1],
                        "CQAVQE Energy Diff": energy_diffs[gate_count-1],
                        "CQAVQE Energy change": energy_changes[gate_count-1],
                        "Mean Evals": mean_evals,
                        "Exact E": exact_e,
                        "Min E": min_e,
                        "Delta min E": delta_min_e,
                        "Median E": median_e,
                        "Delta median E": delta_median_e}
            
            all_data.append(data_dict)

        df = pd.DataFrame(all_data).sort_values("Delta median E", ascending=False)
        df.to_excel(os.path.join(folder_path, "ansatz_steps.xlsx"), index=False)

Running for AHO potential and 2 cutoff
Running for AHO potential and 4 cutoff
Running for AHO potential and 8 cutoff
Running for AHO potential and 16 cutoff
Running for AHO potential and 32 cutoff
Running for AHO potential and 64 cutoff
Running for DW potential and 2 cutoff
Running for DW potential and 4 cutoff
Running for DW potential and 8 cutoff
Running for DW potential and 16 cutoff
Running for DW potential and 32 cutoff
Running for DW potential and 64 cutoff


In [5]:
df = df.sort_values("avqe step", ascending=True)
df

,gate list,avqe step,Converged Runs,step_energies,CQAVQE Energy Diff,CQAVQE Energy change,Mean Evals,Exact E,Min E,Delta min E,Median E,Delta median E
0,RY[$q_0$],1,100/100,1.093911,0.202279,0.000000e+00,4584,0.891632,-0.016554,0.908186,0.443462,0.448170
11,"RY[$q_0$], CRY[$q_0$, $q_1$]",2,100/100,0.987155,0.095523,1.067558e-01,7937,0.891632,-0.015852,0.907485,0.332803,0.558829
20,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$]",3,100/100,0.974805,0.083173,1.235004e-02,10338,0.891632,-0.016547,0.908179,0.500335,0.391297
21,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",4,100/100,0.918832,0.027200,5.597322e-02,12713,0.891632,-0.016667,0.908299,0.474839,0.416793
22,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",5,100/100,0.916445,0.024813,2.387085e-03,15705,0.891632,-0.016501,0.908133,0.812649,0.078983
23,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",6,100/100,0.906359,0.014727,1.008558e-02,18277,0.891632,-0.016661,0.908293,0.607111,0.284521
24,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",7,100/100,0.905997,0.014365,3.621511e-04,20409,0.891632,-0.013503,0.905135,2.806068,1.914436
25,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",8,100/100,0.904131,0.012498,1.866385e-03,23125,0.891632,-0.009518,0.901151,1.993062,1.101429
26,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",9,100/100,0.892808,0.001175,1.132317e-02,25627,0.891632,-0.016642,0.908274,0.827061,0.064571
1,"RY[$q_0$], CRY[$q_0$, $q_1$], RY[$q_2$], RY[$q...",10,100/100,0.892529,0.000896,2.788128e-04,27747,0.891632,-0.012406,0.904038,2.306649,1.415017


In [8]:
potentials = ['DW']#,'AHO']
cutoffs = [8]

for potential in potentials:
    for cutoff in cutoffs:

        print(f"Running for {potential} potential and {str(cutoff)} cutoff")

        folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\AnsatzSteps2\10000\{}\{}".format(potential,cutoff))
        folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

        num_qubits = int(np.log2(cutoff)+1) 
        ansatz_name = f"CQAVQE_{potential}{cutoff}_exact"
        ansatz = ansatze.get(ansatz_name)
        num_params = ansatz.n_params

        params = np.array([0]*num_params)
        gates = gate_list_from_ansatz(ansatz, params, num_qubits)

        gate_list = {}
        op_labels = ""
        counter = 1
        for item in gates:
            if item["gate"] == "BasisState":
                continue

            gate = item["gate"]
            wires = item["wires"]

            qiskit_wires = [f"$q_{(num_qubits - 1) - w}$" for w in wires]
            label = f"{gate}[{', '.join(qiskit_wires)}]"

            if counter == 1:
                op_labels = label
            else:
                op_labels = op_labels + ", " + label

            gate_list[str(counter)] = op_labels
            counter +=1

        avqe_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles\{}\data_{}.txt".format(potential,cutoff))

        with open(avqe_path, "r", encoding="utf-8") as f:
            text = f.read()

        json_part = text[: text.rfind("}") + 1]
        data = json.loads(json_part)

        energy_diffs = data["energy_diffs"]
        energy_changes = data["energy_change"]
        step_energies = data["best_energy_list_reduced"]

        all_data = []

        for f in folders:

            gate_count = label_to_gate_count(f)

            data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
            
            with open(data_path, 'r') as file:
                data = json.load(file)

            success = sum(data['success'])
            exact_e = np.min(data['exact_eigenvalues'])
            converged_runs = sum(data['success'])
            mean_evals = int(np.round(np.mean(data['num_evaluations'])))

            min_e = np.min(data['results'])
            delta_min_e = abs(exact_e - np.min(data['results']))
            median_e = np.median(data['results'])
            delta_median_e = abs(exact_e - np.median(data['results']))
            
            time_taken = data['total_VQE_time']

            ansatz_step = gate_list[str(gate_count)]

            data_dict = {"Potential": potential,
                        r"$\Lambda$": cutoff,
                        "avqe step": gate_count,
                        "gate list": ansatz_step, 
                        #'Converged Runs': f"{converged_runs}/{len(data['results'])}", 
                        "VQE-None": step_energies[gate_count-1],
                        "VQE-None-Diff": energy_diffs[gate_count-1],
                        #"CQAVQE Energy change": energy_changes[gate_count-1],
                        #"Mean Evals": mean_evals,
                        #"Min E": min_e,
                        #"Delta min E": delta_min_e,
                        "VQE-10k": median_e,
                        "VQE-10K-Diff": delta_median_e,
                        #"Exact E": exact_e,
                        }
            
            all_data.append(data_dict)

        df = pd.DataFrame(all_data).sort_values("avqe step", ascending=True)
        df.to_latex(os.path.join(folder_path, f"{potential}{cutoff}.tex"), index=False, float_format="%.6f")

Running for DW potential and 8 cutoff
